<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Spark/blob/master/Aula_1_Introdu%C3%A7%C3%A3o_aos_Dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.0.1 with hadoop 2.7 and Java 8. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Run a local spark session to test your installation:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

# Reading a CSV from google drive

Utilizando o Google Colab, é possível importar os datasets diretamente do Google Drive, sem ter que realizar o upload manual dos mesmos para a instância colab manualmente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spark.read\
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("delimiter", ",") \
  .csv("drive/My\ Drive/My\ Professional\ Carrer/Spark\ course/virtual_classroom/colab_test/test.csv") \
  .show()

# Introdução aos Dataframes
Nessa aula, daremos uma breve explicação do que vêm a ser **Dataframes** no âmbito do Apache Spark e como eles podem ser muito úteis na construção de aplicações escaláveis. Construiremos um dataframe baseado em um arquivo .csv de testes.

### Overwiew de uma aplicação Spark
Relembrando o conceito de uma aplicação Spark submetida a um cluster
![Spark](https://spark.apache.org/docs/latest/img/cluster-overview.png)
*Fonte: https://spark.apache.org/docs/latest/cluster-overview.html*

Podemos ver aqui dois processos principais:

- Executors: processo responsável por realizar a computação bruta dos dados, de forma paralela, ou seja, cada executor em um cluster irá processar uma parte dos dados.
- Driver: processo responsável por coordenar a aplicação e enviar instrução de processamento (transformações) para os executores e receber os resultados das ações. **Códigos não paralelizáveis (códigos que não usufruem dos Dataframes ou RDDs), consumirão poder de processamento e memória do processo Driver**. Exemplo: Pandas

### SparkSession
O `SparkSession` é o objeto responsável por criar Dataframes, ou seja, estrutura de dados paralelizáveis semelhante aos dataframes do framework `pandas` e da linguagem `R`. Em modos interativos, como é o caso do CLI pyspark, o objeto `SparkSession` já vem previamente instânciado, permitindo ao usuário, o uso imediato. Para checar, basta rodar o comando abaixo:


In [ ]:
spark

### Criando um Spark Dataframe a partir de um array de dicionários

In [ ]:
data_dict = [{
    "nome":"Jose",
    "idade": 52,
    "profissao": "Advogado"
},{
    "nome":"Maria",
    "idade": 35,
    "profissao": "Médico"
},{
    "nome":"Pedro",
    "idade": 25,
    "profissao": "Estudante"
}]

data_dict

Método do SparkSession, createDataFrame responsável por criar o DataFrame


In [ ]:
df = spark.createDataFrame(data_dict)

Método show() responsável por exibir o DataFrame

In [ ]:
df.show()

+-----+-----+---------+
|idade| nome|profissao|
+-----+-----+---------+
|   52| Jose| Advogado|
|   35|Maria|   Médico|
|   25|Pedro|Estudante|
+-----+-----+---------+



Printando o schema do DataFrame

In [ ]:
df.printSchema()

root
 |-- idade: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- profissao: string (nullable = true)



## Criando um dataframe a partir de um arquivo CSV

Utilizaremos um dataset que contém algumas das vendas de jogos para consoles. O mesmo se encontra em https://www.kaggle.com/gregorut/videogamesales/data.

Obs.: é necessário criar uma conta no Kaggle para isso.

Realizado o download, importe-o para dentro do armazenamento no seu notebook. Todos os arquivos de dados dentro de uma seção irão ser removidos após a finalização da mesma. Caso deseje salvar permanentemente os arquivos de dados, utilize a opção de importar do Google Drive (seção "Reading a CSV from google drive").

In [ ]:
data_dir = "vgsales.csv"

### Sem inferência de schema


In [ ]:
df = spark.read.load(data_dir, format="csv", header="true")

Visualizando o Dataframe Lido

In [ ]:
df.show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|          33|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|   11.27|    8.89|   10.22|  

Visualizando o Schema

In [ ]:
df.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: string (nullable = true)
 |-- EU_Sales: string (nullable = true)
 |-- JP_Sales: string (nullable = true)
 |-- Other_Sales: string (nullable = true)
 |-- Global_Sales: string (nullable = true)



### Usando a opção para inferir o schema dos dados automaticamente

In [ ]:
df = spark.read.load(data_dir, format="csv", inferSchema="true", header="true")

In [ ]:
df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



### Utilizando um schema definido pelo usuário

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("new_rank", IntegerType(), True),
    StructField("new_name", StringType(), True),
    StructField("new_platform", StringType(), True),
    StructField("new_year", IntegerType(), True),
    StructField("new_genre", StringType(), True),
    StructField("new_publisher", StringType(), True),
    StructField("new_na_sales", DoubleType(), True),
    StructField("new_eu_sales", DoubleType(), True),
    StructField("new_jp_sales", DoubleType(), True),
    StructField("new_other_sales", DoubleType(), True),
    StructField("new_global_sales", DoubleType(), True)
])

In [ ]:
df = spark.read.load(data_dir, format="csv", schema=schema, header="true")

In [ ]:
df.printSchema()

root
 |-- new_rank: integer (nullable = true)
 |-- new_name: string (nullable = true)
 |-- new_platform: string (nullable = true)
 |-- new_year: integer (nullable = true)
 |-- new_genre: string (nullable = true)
 |-- new_publisher: string (nullable = true)
 |-- new_na_sales: double (nullable = true)
 |-- new_eu_sales: double (nullable = true)
 |-- new_jp_sales: double (nullable = true)
 |-- new_other_sales: double (nullable = true)
 |-- new_global_sales: double (nullable = true)



In [ ]:
df.show()

+--------+--------------------+------------+--------+------------+--------------------+------------+------------+------------+---------------+----------------+
|new_rank|            new_name|new_platform|new_year|   new_genre|       new_publisher|new_na_sales|new_eu_sales|new_jp_sales|new_other_sales|new_global_sales|
+--------+--------------------+------------+--------+------------+--------------------+------------+------------+------------+---------------+----------------+
|       1|          Wii Sports|         Wii|    2006|      Sports|            Nintendo|       41.49|       29.02|        3.77|           8.46|           82.74|
|       2|   Super Mario Bros.|         NES|    1985|    Platform|            Nintendo|       29.08|        3.58|        6.81|           0.77|           40.24|
|       3|      Mario Kart Wii|         Wii|    2008|      Racing|            Nintendo|       15.85|       12.88|        3.79|           3.31|           35.82|
|       4|   Wii Sports Resort|         

## Formatos suportados
O Spark é capaz de ler uma gama de arquivos e fontes de dados nativamente. Dentre eles, temos:

- CSV
- JSON (JSON lines text format)
- Parquet files
- ORC files
- Hive tables
- JDBC (MySQL, Postgresql, Oracle, SAP, e inúmeros outros BDs)
- Avro

É possível também encontrar dependências de terceiros, que apliam ainda mais o leque de opções de fontes de dados para o Spark. Ler mais em https://spark.apache.org/third-party-projects.html

# Exercícios
Para iniciar os exercícios, configure a variável abaixo para que possa apontar para o caminho correto em seu ambiente de desenvolvimento:


In [ ]:
data_dir = "vgsales.csv"

Para auxiliar na visualização dos resultados, execute o paragrafo abaixo para definir a função display_df():

In [ ]:
def display_df(df, n=20):
    return df.limit(n).toPandas()

In [ ]:
display_df(df)
#df.show()

1) Leia o arquivo vgsales.csv em formato de inferência de schema e header desativado:

In [ ]:
df = spark.read.load(data_dir, format="csv", inferSchema="true", header="false")
df.show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
| _c0|                 _c1|     _c2| _c3|         _c4|                 _c5|     _c6|     _c7|     _c8|        _c9|        _c10|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|  

2) Leia o arquivo vgsales.csv com um schema pré-definido e header ativado:

Obs.: considere todos os valores numéricos como inteiros para esse exercício


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("new_rank", IntegerType(), True),
    StructField("new_name", StringType(), True),
    StructField("new_platform", StringType(), True),
    StructField("new_year", StringType(), True),
    StructField("new_genre", StringType(), True),
    StructField("new_publisher", StringType(), True),
    StructField("new_na_sales", IntegerType(), True),
    StructField("new_eu_sales", IntegerType(), True),
    StructField("new_jp_sales", IntegerType(), True),
    StructField("new_other_sales", IntegerType(), True),
    StructField("new_global_sales", IntegerType(), True)
])

df = spark.read.load(data_dir, format="csv", schema=schema, header="true")
df.printSchema()

root
 |-- new_rank: integer (nullable = true)
 |-- new_name: string (nullable = true)
 |-- new_platform: string (nullable = true)
 |-- new_year: string (nullable = true)
 |-- new_genre: string (nullable = true)
 |-- new_publisher: string (nullable = true)
 |-- new_na_sales: integer (nullable = true)
 |-- new_eu_sales: integer (nullable = true)
 |-- new_jp_sales: integer (nullable = true)
 |-- new_other_sales: integer (nullable = true)
 |-- new_global_sales: integer (nullable = true)

+--------+--------------------+------------+--------+------------+--------------------+------------+------------+------------+---------------+----------------+
|new_rank|            new_name|new_platform|new_year|   new_genre|       new_publisher|new_na_sales|new_eu_sales|new_jp_sales|new_other_sales|new_global_sales|
+--------+--------------------+------------+--------+------------+--------------------+------------+------------+------------+---------------+----------------+
|       1|          Wii Sports|